In [1]:
#/home/agupta/pretrained
#!pip install keras
import numpy as np
import pandas as pd 
import re
from bs4 import BeautifulSoup
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Concatenate, TimeDistributed
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import warnings
pd.set_option("display.max_colwidth", 200)
warnings.filterwarnings("ignore")

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/agupta/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
!pip install bs4

In [3]:
#from shutil import copyfile

# copy our file into the working directory (make sure it has .py suffix)
#copyfile(src = "/kaggle/input/attentionfile/attention.py", dst = "/kaggle/working/attention.py")




from attention import *

In [4]:
import re
import pandas as pd
def clean_str(sentence):
    sentence=re.sub("[#.]+","#",sentence)
    return sentence
def get_text_list(data_path,toy):
    with open(data_path,"r",encoding="utf-8") as f:
        if not toy:
            return [clean_str(x.strip()) for x in f.readlines()]
        else:
            return [clean_str(x.strip()) for x in f.readlines()][:500]

In [5]:
text=get_text_list('train.article.txt',True)

summary=get_text_list('train.title.txt',True)

In [6]:
d = {'Text':text,'Summary':summary}
data=pd.DataFrame(d)

In [7]:
data.dropna(axis=0,inplace=True)
data

,Text,Summary
0,"australia 's current account deficit shrunk by a record # billion dollars -lrb- # billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed #",australian current account deficit narrows sharply
1,"at least two people were killed in a suspected bomb attack on a passenger bus in the strife-torn southern philippines on monday , the military said #",at least two dead in southern philippines blast
2,"australian shares closed down # percent monday following a weak lead from the united states and lower commodity prices , dealers said #",australian stocks close down # percent
3,south korea 's nuclear envoy kim sook urged north korea monday to restart work to disable its nuclear plants and stop its `` typical '' brinkmanship in negotiations #,envoy urges north korea to restart nuclear disablement
4,"south korea on monday announced sweeping tax reforms , including income and corporate tax cuts to boost growth by stimulating sluggish private consumption and business investment #",skorea announces tax cuts to stimulate economy
...,...,...
495,"bollywood legend amitabh bachchan 's new film `` the last lear '' opens on friday , the latest in a string of english language movies to hit the screens as indian cinema explores new themes and co...",bollywood talks english beginning with the bard
496,"chinese share prices were lower in early thursday trade led by financial companies as sentiment remained poor amid worries over slowing corporate earnings , dealers said #",chinese shares lower in morning trade
497,"alaska officials fired barbs at state governor sarah palin wednesday , trying to burst the bubble the republican party has been riding on since her surprise nomination two weeks ago as john mccain...",obama 's alaska <unk> seek to burst the palin bubble
498,john mccain and barack obama were to observe a truce thursday in their increasingly bitter white house contest with a joint appearance at ground zero on the seventh anniversary of the #\/# attacks #,white house combatants observe ground zero truce


In [8]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",
                           "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",
                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",
                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",
                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",
                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",
                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",
                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",
                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",
                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",
                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",
                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",
                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",
                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",
                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",
                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
                           "you're": "you are", "you've": "you have"}

In [9]:
stop_words = set(stopwords.words('english')) 

def text_cleaner(text,num):
    newString = text.lower()
    newString = BeautifulSoup(newString, "lxml").text
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    newString = re.sub('[m]{2,}', 'mm', newString)
    if(num==0):
        tokens = [w for w in newString.split() if not w in stop_words]
    else:
        tokens=newString.split()
    long_words=[]
    for i in tokens:
        if len(i)>1:                                                 #removing short word
            long_words.append(i)   
    return (" ".join(long_words)).strip()

In [10]:
#call the function
cleaned_text = []
for t in data['Text']:
    cleaned_text.append(text_cleaner(t,0))

In [11]:
cleaned_text[:5]

['australia current account deficit shrunk record billion dollars lrb billion us rrb june quarter due soaring commodity prices figures released monday showed',
 'least two people killed suspected bomb attack passenger bus strife torn southern philippines monday military said',
 'australian shares closed percent monday following weak lead united states lower commodity prices dealers said',
 'south korea nuclear envoy kim sook urged north korea monday restart work disable nuclear plants stop typical brinkmanship negotiations',
 'south korea monday announced sweeping tax reforms including income corporate tax cuts boost growth stimulating sluggish private consumption business investment']

In [12]:
#call the function
cleaned_summary = []
for t in data['Summary']:
    cleaned_summary.append(text_cleaner(t,1))

In [13]:
cleaned_summary[:10]

['australian current account deficit narrows sharply',
 'at least two dead in southern philippines blast',
 'australian stocks close down percent',
 'envoy urges north korea to restart nuclear disablement',
 'skorea announces tax cuts to stimulate economy',
 'taiwan shares close down percent',
 'australian stocks close down percent',
 'spain colonial posts billion euro loss',
 'kadhafi promises wide political economic reforms',
 'un top aid official arrives in drought hit ethiopia']

In [14]:

data['cleaned_text']=cleaned_text
data['cleaned_summary']=cleaned_summary
data

,Text,Summary,cleaned_text,cleaned_summary
0,"australia 's current account deficit shrunk by a record # billion dollars -lrb- # billion us -rrb- in the june quarter due to soaring commodity prices , figures released monday showed #",australian current account deficit narrows sharply,australia current account deficit shrunk record billion dollars lrb billion us rrb june quarter due soaring commodity prices figures released monday showed,australian current account deficit narrows sharply
1,"at least two people were killed in a suspected bomb attack on a passenger bus in the strife-torn southern philippines on monday , the military said #",at least two dead in southern philippines blast,least two people killed suspected bomb attack passenger bus strife torn southern philippines monday military said,at least two dead in southern philippines blast
2,"australian shares closed down # percent monday following a weak lead from the united states and lower commodity prices , dealers said #",australian stocks close down # percent,australian shares closed percent monday following weak lead united states lower commodity prices dealers said,australian stocks close down percent
3,south korea 's nuclear envoy kim sook urged north korea monday to restart work to disable its nuclear plants and stop its `` typical '' brinkmanship in negotiations #,envoy urges north korea to restart nuclear disablement,south korea nuclear envoy kim sook urged north korea monday restart work disable nuclear plants stop typical brinkmanship negotiations,envoy urges north korea to restart nuclear disablement
4,"south korea on monday announced sweeping tax reforms , including income and corporate tax cuts to boost growth by stimulating sluggish private consumption and business investment #",skorea announces tax cuts to stimulate economy,south korea monday announced sweeping tax reforms including income corporate tax cuts boost growth stimulating sluggish private consumption business investment,skorea announces tax cuts to stimulate economy
...,...,...,...,...
495,"bollywood legend amitabh bachchan 's new film `` the last lear '' opens on friday , the latest in a string of english language movies to hit the screens as indian cinema explores new themes and co...",bollywood talks english beginning with the bard,bollywood legend amitabh bachchan new film last lear opens friday latest string english language movies hit screens indian cinema explores new themes content,bollywood talks english beginning with the bard
496,"chinese share prices were lower in early thursday trade led by financial companies as sentiment remained poor amid worries over slowing corporate earnings , dealers said #",chinese shares lower in morning trade,chinese share prices lower early thursday trade led financial companies sentiment remained poor amid worries slowing corporate earnings dealers said,chinese shares lower in morning trade
497,"alaska officials fired barbs at state governor sarah palin wednesday , trying to burst the bubble the republican party has been riding on since her surprise nomination two weeks ago as john mccain...",obama 's alaska <unk> seek to burst the palin bubble,alaska officials fired barbs state governor sarah palin wednesday trying burst bubble republican party riding since surprise nomination two weeks ago john mccain running mate,obama alaska seek to burst the palin bubble
498,john mccain and barack obama were to observe a truce thursday in their increasingly bitter white house contest with a joint appearance at ground zero on the seventh anniversary of the #\/# attacks #,white house combatants observe ground zero truce,john mccain barack obama observe truce thursday increasingly bitter white house contest joint appearance ground zero seventh anniversary attacks,white house combatants observe ground zero truce


In [15]:
max_text_len=50
max_summary_len=15

In [16]:
cnt=0
for i in data['cleaned_text']:
    if(len(i.split())<=30):
        cnt=cnt+1
print(cnt/len(data['cleaned_text']))

1.0


In [17]:
cleaned_text =np.array(data['cleaned_text'])
cleaned_summary=np.array(data['cleaned_summary'])

short_text=[]
short_summary=[]

for i in range(len(cleaned_text)):
    if(len(cleaned_summary[i].split())<=max_summary_len and len(cleaned_text[i].split())<=max_text_len):
        short_text.append(cleaned_text[i])
        short_summary.append(cleaned_summary[i])
        
df=pd.DataFrame({'text':short_text,'summary':short_summary})

In [18]:
df['summary'] = df['summary'].apply(lambda x : 'sostok '+ x + ' eostok')
df

,text,summary
0,australia current account deficit shrunk record billion dollars lrb billion us rrb june quarter due soaring commodity prices figures released monday showed,sostok australian current account deficit narrows sharply eostok
1,least two people killed suspected bomb attack passenger bus strife torn southern philippines monday military said,sostok at least two dead in southern philippines blast eostok
2,australian shares closed percent monday following weak lead united states lower commodity prices dealers said,sostok australian stocks close down percent eostok
3,south korea nuclear envoy kim sook urged north korea monday restart work disable nuclear plants stop typical brinkmanship negotiations,sostok envoy urges north korea to restart nuclear disablement eostok
4,south korea monday announced sweeping tax reforms including income corporate tax cuts boost growth stimulating sluggish private consumption business investment,sostok skorea announces tax cuts to stimulate economy eostok
...,...,...
495,bollywood legend amitabh bachchan new film last lear opens friday latest string english language movies hit screens indian cinema explores new themes content,sostok bollywood talks english beginning with the bard eostok
496,chinese share prices lower early thursday trade led financial companies sentiment remained poor amid worries slowing corporate earnings dealers said,sostok chinese shares lower in morning trade eostok
497,alaska officials fired barbs state governor sarah palin wednesday trying burst bubble republican party riding since surprise nomination two weeks ago john mccain running mate,sostok obama alaska seek to burst the palin bubble eostok
498,john mccain barack obama observe truce thursday increasingly bitter white house contest joint appearance ground zero seventh anniversary attacks,sostok white house combatants observe ground zero truce eostok


In [19]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(np.array(df['text']),np.array(df['summary']),test_size=0.1,random_state=0,shuffle=True)



In [20]:
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences

#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer() 
x_tokenizer.fit_on_texts(list(x_tr))


In [21]:
thresh=4

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in x_tokenizer.word_counts.items():
    #print(value)
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
print(tot_freq)   
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

8467
% of rare words in vocabulary: 78.81558028616851
Total Coverage of rare words: 35.33719144915555


In [22]:
#prepare a tokenizer for reviews on training data
x_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
x_tokenizer.fit_on_texts(list(x_tr))

#convert text sequences into integer sequences
x_tr_seq    =   x_tokenizer.texts_to_sequences(x_tr) 
x_val_seq   =   x_tokenizer.texts_to_sequences(x_val)

#padding zero upto maximum length
x_tr    =   pad_sequences(x_tr_seq,  maxlen=max_text_len, padding='post')
x_val   =   pad_sequences(x_val_seq, maxlen=max_text_len, padding='post')

#size of vocabulary ( +1 for padding token)
x_voc   =  x_tokenizer.num_words + 1
x_voc

534

In [23]:
x_voc

534

In [24]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer()   
y_tokenizer.fit_on_texts(list(y_tr))

In [25]:
thresh=6

cnt=0
tot_cnt=0
freq=0
tot_freq=0

for key,value in y_tokenizer.word_counts.items():
    tot_cnt=tot_cnt+1
    tot_freq=tot_freq+value
    if(value<thresh):
        cnt=cnt+1
        freq=freq+value
    
print("% of rare words in vocabulary:",(cnt/tot_cnt)*100)
print("Total Coverage of rare words:",(freq/tot_freq)*100)

% of rare words in vocabulary: 89.58509737510583
Total Coverage of rare words: 42.422814046665096


In [26]:
#prepare a tokenizer for reviews on training data
y_tokenizer = Tokenizer(num_words=tot_cnt-cnt) 
y_tokenizer.fit_on_texts(list(y_tr))

#convert text sequences into integer sequences
y_tr_seq    =   y_tokenizer.texts_to_sequences(y_tr) 
y_val_seq   =   y_tokenizer.texts_to_sequences(y_val) 

#padding zero upto maximum length
y_tr    =   pad_sequences(y_tr_seq, maxlen=max_summary_len, padding='post')
y_val   =   pad_sequences(y_val_seq, maxlen=max_summary_len, padding='post')

#size of vocabulary
y_voc  =   y_tokenizer.num_words +1

In [27]:
y_tokenizer.word_counts['sostok'],len(y_tr)
x_tr

array([[ 20, 276, 405, ...,   0,   0,   0],
       [145, 406,  30, ...,   0,   0,   0],
       [ 85, 408, 278, ...,   0,   0,   0],
       ...,
       [386,  18, 133, ...,   0,   0,   0],
       [  4,  21, 212, ...,   0,   0,   0],
       [192,  76,   2, ...,   0,   0,   0]], dtype=int32)

In [28]:
ind=[]
for i in range(len(y_tr)):
    cnt=0
    for j in y_tr[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_tr=np.delete(y_tr,ind, axis=0)
x_tr=np.delete(x_tr,ind, axis=0)


In [29]:
ind=[]
for i in range(len(y_val)):
    cnt=0
    for j in y_val[i]:
        if j!=0:
            cnt=cnt+1
    if(cnt==2):
        ind.append(i)

y_val=np.delete(y_val,ind, axis=0)
x_val=np.delete(x_val,ind, axis=0)

In [31]:
from keras import backend as K 
K.clear_session()

latent_dim = 300
embedding_dim=100

# Encoder
encoder_inputs = Input(shape=(max_text_len,))

#embedding layer
enc_emb =  Embedding(x_voc, embedding_dim,trainable=True)(encoder_inputs)

#encoder lstm 1
encoder_lstm1 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output1, state_h1, state_c1 = encoder_lstm1(enc_emb)

#encoder lstm 2
encoder_lstm2 = LSTM(latent_dim,return_sequences=True,return_state=True,dropout=0.4,recurrent_dropout=0.4)
encoder_output2, state_h2, state_c2 = encoder_lstm2(encoder_output1)

#encoder lstm 3
encoder_lstm3=LSTM(latent_dim, return_state=True, return_sequences=True,dropout=0.4,recurrent_dropout=0.4)
encoder_outputs, state_h, state_c= encoder_lstm3(encoder_output2)

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

#embedding layer
dec_emb_layer = Embedding(y_voc, embedding_dim,trainable=True)
dec_emb = dec_emb_layer(decoder_inputs)

decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True,dropout=0.4,recurrent_dropout=0.2)
decoder_outputs,decoder_fwd_state, decoder_back_state = decoder_lstm(dec_emb,initial_state=[state_h, state_c])

# Attention layer
attn_layer = AttentionLayer(name='attention_layer')
attn_out, attn_states = attn_layer([encoder_outputs, decoder_outputs])

# Concat attention input and decoder LSTM output
decoder_concat_input = Concatenate(axis=-1, name='concat_layer')([decoder_outputs, attn_out])

#dense layer
decoder_dense =  TimeDistributed(Dense(y_voc, activation='softmax'))
decoder_outputs = decoder_dense(decoder_concat_input)

# Define the model 
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.summary()

AttributeError: 'google.protobuf.pyext._message.RepeatedCompositeCo' object has no attribute 'append'

In [ ]:
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=2)
checkpoint = ModelCheckpoint('model.h5', monitor='val_loss', save_best_only=True)


In [ ]:
history=model.fit([x_tr,y_tr[:,:-1]], y_tr.reshape(y_tr.shape[0],y_tr.shape[1], 1)[:,1:] ,
                  epochs=50,callbacks=[es,checkpoint],batch_size=128, validation_data=([x_val,y_val[:,:-1]], 
                                                                            y_val.reshape(y_val.shape[0],y_val.shape[1], 1)[:,1:]))

In [ ]:
from matplotlib import pyplot
pyplot.plot(history.history['loss'], label='train')
pyplot.plot(history.history['val_loss'], label='test')
pyplot.legend()
pyplot.show()

In [ ]:
reverse_target_word_index=y_tokenizer.index_word
reverse_source_word_index=x_tokenizer.index_word
target_word_index=y_tokenizer.word_index

In [ ]:
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    e_out, e_h, e_c = encoder_model.predict(input_seq)
    
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    
    # Populate the first word of target sequence with the start word.
    target_seq[0, 0] = target_word_index['sostok']

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
      
        output_tokens, h, c = decoder_model.predict([target_seq] + [e_out, e_h, e_c])

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token = reverse_target_word_index[sampled_token_index]
        
        if(sampled_token!='eostok'):
            decoded_sentence += ' '+sampled_token

        # Exit condition: either hit max length or find stop word.
        if (sampled_token == 'eostok'  or len(decoded_sentence.split()) >= (max_summary_len-1)):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update internal states
        e_h, e_c = h, c

    return decoded_sentence

In [ ]:
def seq2summary(input_seq):
    newString=''
    for i in input_seq:
        if((i!=0 and i!=target_word_index['sostok']) and i!=target_word_index['eostok']):
            newString=newString+reverse_target_word_index[i]+' '
    return newString

def seq2text(input_seq):
    newString=''
    for i in input_seq:
        if(i!=0):
            newString=newString+reverse_source_word_index[i]+' '
    return newString

In [ ]:
model.load_weights('model.h5')
# Encode the input sequence to get the feature vector
encoder_model = Model(inputs=encoder_inputs,outputs=[encoder_outputs, state_h, state_c])

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_hidden_state_input = Input(shape=(max_text_len,latent_dim))

# Get the embeddings of the decoder sequence
dec_emb2= dec_emb_layer(decoder_inputs) 
# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=[decoder_state_input_h, decoder_state_input_c])

#attention inference
attn_out_inf, attn_states_inf = attn_layer([decoder_hidden_state_input, decoder_outputs2])
decoder_inf_concat = Concatenate(axis=-1, name='concat')([decoder_outputs2, attn_out_inf])

# A dense softmax layer to generate prob dist. over the target vocabulary
decoder_outputs2 = decoder_dense(decoder_inf_concat) 

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + [decoder_hidden_state_input,decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs2] + [state_h2, state_c2])

In [ ]:
for i in range(0,100):
    print("Review:",seq2text(x_tr[i]))
    print("Original summary:",seq2summary(y_tr[i]))
    print("Predicted summary:",decode_sequence(x_tr[i].reshape(1,max_text_len)))
    print("\n")

In [ ]:
pd.DataFrame(history.history).to_csv("loss.csv")